# Parallel QAOA Portfolio Optimization with Domain Wall Encoding (DWE)

This notebook demonstrates how to run multiple QAOA optimization trajectories in parallel using Python's `multiprocessing` module. This is particularly useful for finding better solutions in complex, non-convex landscapes like that of QAOA, and for speeding up the overall parameter search.

We will leverage your existing Domain Wall Encoding (DWE) problem setup, which ensures the Hamming-weight constraint is handled as a penalty within the objective function.

**Important Note on Multiprocessing in Jupyter:**
The `AttributeError` you were encountering when using `ProcessPoolExecutor` in Jupyter is a common issue due to how child processes handle imports from the notebook's interactive `__main__` scope. The solution implemented here involves moving the core multiprocessing target functions (`run_single_optimization` and `minimize_nlopt`) into a separate Python file (`qaoa_utils.py`), which is the most robust way to ensure they are discoverable by spawned processes.

## 1. Setup and Imports

First, we'll set up all the necessary imports. Note the new imports from `qaoa_utils` and `functools`.

In [ ]:
import numpy as np
import os 
import time 
import nlopt 
import multiprocessing 
from concurrent.futures import ProcessPoolExecutor 
from functools import partial # NEW: For passing fixed arguments to mapped function

# NEW: Import the functions from the external file
from qaoa_utils import minimize_nlopt, run_single_optimization

from qiskit_aer import AerSimulator
from qokit.portfolio_optimization import get_sk_ini
from qokit.qaoa_objective_portfolio import get_qaoa_portfolio_objective

# Optional: for later analysis of optimal bitstring
from qiskit.circuit import ParameterVector
from qiskit.primitives import Sampler
from qokit.qaoa_circuit_portfolio import get_parameterized_qaoa_circuit

## 2. Define Problem Parameters (with DWE)

This section mirrors your existing notebook's setup for constructing the `po_problem` dictionary, which contains the `J` and `h` coefficients after applying the DWE penalty. This is crucial for maintaining your DWE approach.

In [ ]:
## Problem Statement & Setup for Dicke + DWE Technique

# Define problem parameters
N = 25
# Number of assets. Change N to your desired value (e.g., 20, 25, 30)
K = int(N * 0.4) # Example: select 40% of assets. Adjust as needed, ensure K < N.
p = 1 # Number of QAOA layers (start with p=1, higher p will be much slower for simulation)

# --- Generate random mu and Sigma for larger N (as you won't have specific data) ---
np.random.seed(42) # for reproducibility of the problem definition
mu = np.random.uniform(0.05, 0.20, N)
# Generate a random symmetric positive semi-definite matrix for Sigma
Sigma = np.random.uniform(0.001, 0.015, (N, N))
Sigma = (Sigma + Sigma.T) / 2 # Make it symmetric
# Add a small diagonal component to ensure it's positive semi-definite (for stability)
Sigma = Sigma + np.eye(N) * 0.005

q = 0.5 # Risk aversion parameter
lambda_sum = 100 # DWE-inspired penalty coefficient for sum constraint

# --- Calculate Coefficients for Total Cost Hamiltonian (H_C = H_C^objective + H_C^DWE-penalty) ---
# This logic is from your previous detailed plan
factor_J_obj = (2 * q) / (K**2) 
factor_h_linear_obj = -1 / K
factor_h_diagonal_obj = q / (K**2) 

J_coeffs_objective = {}
h_coeffs_objective = {}

for i in range(N):
    for j in range(i + 1, N):
        J_coeffs_objective[(i, j)] = factor_J_obj * Sigma[i, j]

for i in range(N):
    h_coeffs_objective[i] = factor_h_linear_obj * mu[i] + factor_h_diagonal_obj * Sigma[i, i]

J_coeffs_total = {} # J_coeffs_total calculated here
h_coeffs_total = {} # h_coeffs_total calculated here

for (i, j), val in J_coeffs_objective.items():
    J_coeffs_total[(i, j)] = val + 2 * lambda_sum

for i, val in h_coeffs_objective.items():
    h_coeffs_total[i] = val - 5 * lambda_sum

# Construct the custom po_problem dictionary with DWE coefficients
po_problem = {
    "N": N,
    "K": K,
    "q": q, 
    "J": J_coeffs_total, # This is the DWE-transformed QUBO J
    "h": h_coeffs_total, # This is the DWE-transformed QUBO h
    "means": mu,         # Storing original for later classical evaluation
    "cov": Sigma,        # Storing original for later classical evaluation
    "q_orig": q,         # Storing original q for later classical evaluation (renamed to avoid conflict with `q` above)
    "scale": 1.0         
}

print(f"--- Problem Parameters & Custom QUBO Defined for N={N} ---")
print(f"Number of assets (N): {N}")
print(f"Assets to select (K): {K}")
print(f"QAOA Layers (p): {p}")
print(f"DWE Penalty Lambda_sum: {lambda_sum}")

# --- Handle best_portfolio (from brute-force) ---
# Brute-force is not feasible for N > 20.
# We will set best_portfolio to None if N > 20, so AR calculation is skipped.
best_portfolio = (None, None) # Default placeholder
if N <= 20: # Example threshold for brute-force feasibility
    try:
        from qokit.portfolio_optimization import portfolio_brute_force
        print("Calculating classical brute-force solution (may take time for N up to 20)...")
        # For brute-force, use a simpler problem definition that doesn't have DWE penalty already
        # The brute-force solver expects the original mu/Sigma/q.
        original_po_problem_for_brute_force = {"N":N, "K":K, "q":q, "means":mu, "cov":Sigma}
        best_portfolio = portfolio_brute_force(original_po_problem_for_brute_force, return_bitstring=False)
        print(f"Brute-force classical optimal energy: {best_portfolio[0]:.6f}")
        print(f"Brute-force classical worst energy: {best_portfolio[1]:.6f}")
    except ImportError:
        print("qokit.portfolio_optimization.portfolio_brute_force not available or failed.")
        print("Approximation Ratio will not be calculated.")
    except Exception as e:
        print(f"Brute-force calculation failed: {e}")
        print("Approximation Ratio will not be calculated.")
else:
    print(f"Brute-force calculation is not feasible for N = {N} > 20. Approximation Ratio will not be calculated.")

print("-" * 50)

## 3. `minimize_nlopt` Function

This helper function is now defined in `qaoa_utils.py` and imported.

In [ ]:
# This function is now defined in qaoa_utils.py and imported at the top.
# You no longer need to define it here.

## 4. `run_single_optimization` Function

This is the core function for parallel execution, now defined in `qaoa_utils.py` and imported. It has been modified to explicitly accept `po_problem` and `best_portfolio` as arguments.

In [ ]:
# This function is now defined in qaoa_utils.py and imported at the top.
# It now takes `po_problem_arg` and `best_portfolio_arg` as explicit arguments.

## 5. Execute Parallel Optimization

This is the main control block. It defines the configurations for each parallel run and uses `ProcessPoolExecutor` to distribute the `run_single_optimization` calls across your CPU cores. The `if __name__ == "__main__":` guard remains crucial for multiprocessing in notebooks.

We use `functools.partial` here to pass the static `po_problem` and `best_portfolio` to each call of `run_single_optimization` in the parallel pool.

In [ ]:
# Initialize these variables outside the if __name__ block
# to ensure they are always defined, even if the multiprocessing part isn't run
# or if the kernel is restarted.
all_successful_results = []
best_overall_params = None # Initialize to None, will be updated if runs complete

if __name__ == "__main__":
    try:
        multiprocessing.set_start_method('spawn', force=True)
        print("Multiprocessing start method set to 'spawn'.")
    except RuntimeError:
        print("Multiprocessing start method already set.")

    num_parallel_runs = 4 # Adjust based on your CPU cores (e.g., os.cpu_count() or os.cpu_count() - 1)
    
    run_configurations = [
        {'seed': 101, 'p': p, 'max_evals': 1},  # <--- ADDED 'max_evals'
        {'seed': 102, 'p': p, 'max_evals': 1},  # <--- ADDED 'max_evals'
        {'seed': 103, 'p': p, 'max_evals': 1},  # <--- You can vary it per run
        {'seed': 104, 'p': p, 'max_evals': 1},  # <--- ADDED 'max_evals'
        # {'seed': 105, 'p': p, 'max_evals': 10}, # <--- ADDED 'max_evals'
        # {'seed': 106, 'p': p, 'max_evals': 10}  # <--- ADDED 'max_evals'
        # Add more configurations as needed, customizing 'max_evals' for each
    ]

    all_successful_results = [] # Reset results list before new runs

    print(f"Starting {len(run_configurations)} parallel QAOA optimization runs...N={N}")
    print(f"Using up to {num_parallel_runs} parallel processes.")

    with ProcessPoolExecutor(max_workers=num_parallel_runs) as executor:
        # Use functools.partial to fix po_problem and best_portfolio as arguments for run_single_optimization
        func_to_map = partial(run_single_optimization, 
                              po_problem_arg=po_problem, 
                              best_portfolio_arg=best_portfolio)
        results_iterator = executor.map(func_to_map, run_configurations)

        for i, result in enumerate(results_iterator):
            if result is not None: 
                all_successful_results.append(result)
                print(f"Collected result for run {result['seed']}: Energy={result['energy']:.8f}, AR={result['ar']}")
            else:
                print(f"Run {i+1} failed or returned no result.")

    print("\n" + "---" * 20)
    print("### Summary of All Successful Parallel Optimization Results ###")
    if all_successful_results:
        all_successful_results.sort(key=lambda x: x['energy'])

        for res in all_successful_results:
            print(f"Seed: {res['seed']}, Energy: {res['energy']:.8f}, AR: {res['ar']}")

        best_overall_energy = all_successful_results[0]['energy']
        best_overall_ar = all_successful_results[0]['ar']
        best_overall_params = all_successful_results[0]['optimized_params'] 
        
        print("\n" + "---" * 20)
        print(f"Best overall energy found: {best_overall_energy:.8f}")
        print(f"Corresponding Approximation Ratio: {best_overall_ar}")
        print(f"Parameters of best run: {best_overall_params}")
        print("---" * 20)
    else:
        print("No successful optimization runs completed.")



## 6. Optional: Analyze the Best Optimal Bitstring

After finding the best parameters from the parallel runs, you might want to run the circuit one more time with those parameters to get the most probable bitstring(s) and their classical energy.

In [ ]:
if __name__ == "__main__" and all_successful_results and best_overall_params is not None:
    print("\n" + "---" * 20)
    print("### Analyzing the Single Best Result for Optimal Bitstring ###")
    
    optimal_parameters = best_overall_params 
    print(f"Using optimal parameters from best run: {optimal_parameters}")

    # --- Extracting the Optimal Bitstring (Measurement) ---
    gamma_opt = ParameterVector('gamma', p)
    beta_opt = ParameterVector('beta', p)

    optimal_circuit = get_parameterized_qaoa_circuit(
        po_problem=po_problem,
        depth=p,
        ini_type='dicke',
        mixer_type='trotter_ring',
        T=1,
        simulator=None,
        mixer_topology='linear',
        gamma=gamma_opt,
        beta=beta_opt
    )

    optimal_circuit_bound = optimal_circuit.assign_parameters({
        gamma_opt: optimal_parameters[:p],
        beta_opt: optimal_parameters[p:]
    })

    optimal_circuit_bound.measure_all()

    sampler_shots = 1024 
    sampler = Sampler()

    print(f"\n--- Sampling for Optimal Bitstring ({sampler_shots} shots) ---")
    sampler_job = sampler.run(optimal_circuit_bound, shots=sampler_shots)
    sampler_result = sampler_job.result()

    counts = sampler_result.quasi_dists[0].binary_probabilities()
    classical_counts = {k: v for k, v in counts.items()} 

    sorted_counts = sorted(classical_counts.items(), key=lambda item: item[1], reverse=True)

    print("Top 5 most frequent bitstrings and their probabilities:")
    optimal_bitstring = None
    for i, (bitstring, probability) in enumerate(sorted_counts[:5]):
        print(f"  {bitstring}: {probability:.4f}")
        if i == 0:
            optimal_bitstring = bitstring

    if optimal_bitstring:
        optimal_selection = np.array([int(b) for b in optimal_bitstring])
        selected_assets_count = np.sum(optimal_selection)

        print(f"\nMost probable portfolio selection (bitstring): {optimal_bitstring}")
        print(f"Number of selected assets: {selected_assets_count} (Expected K={K})")

        def evaluate_bitstring_energy(bitstring_array, original_mu, original_Sigma, original_q):
            x = bitstring_array
            portfolio_variance = np.dot(x, np.dot(original_Sigma, x))
            portfolio_return = np.dot(original_mu, x)
            return original_q * portfolio_variance - portfolio_return

        optimal_classical_energy_for_bitstring = evaluate_bitstring_energy(
            optimal_selection,
            po_problem['means'], 
            po_problem['cov'],   
            po_problem['q_orig'] 
        )
        print(f"Classical objective energy for the most probable bitstring: {optimal_classical_energy_for_bitstring:.6f}")
        print(f"Number of assets selected by most probable bitstring: {np.sum(optimal_selection)}")
    else:
        print("Could not determine optimal bitstring.")
